In [2]:
import os
import logging
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.cache_handler import CacheFileHandler
import pandas as pd

logging.basicConfig(filename='spotipy.log',
                    encoding='utf-8', level=logging.INFO)
logging.basicConfig(format='%(asctime)s %(message)s')
logger = logging.getLogger(__name__)

# Cache
cache_path = '.cache'
cache_handler = CacheFileHandler(cache_path=cache_path)
logger.info(cache_handler.get_cached_token())

# TEST ID's
logger.debug('Test Album: "0ZgTSm1VI55AhE09Nzvv11"')
logger.debug('Test Playlist: "37i9dQZF1DXagUeYbNSnOA"')

# Conexion con la API de Spotify.
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=os.environ.get('SPOTIPY_CLIENT_ID'),
                                                           client_secret=os.environ.get(
                                                               'SPOTIPY_CLIENT_SECRET'),
                                                           cache_handler=cache_handler))
# Meta y Features
meta = ['name', 'album', 'artists',
        'release_date', 'duration_ms', 'popularity']
features = ['danceability', 'acousticness', 'energy', 'instrumentalness', 'liveness',
            'loudness', 'speechiness', 'key', 'mode', 'valence', 'tempo', 'time_signature']


def time_format(ms):
    """ Metodo de conversion de milisegundos a formato MM:SS """
    if int(ms) >= 3600000:  # Más de una hora
        return "{:02}:{:02}:{:02}".format(int((ms / 1000.0) / 3600), int((ms / 1000.0 / 60) % 60),
                                          int(ms / 1000.0 % 60))
    else:
        return "{:02}:{:02}".format(int((ms / 1000.0 / 60) % 60), int(ms / 1000.0 % 60))


def album_get_tracks_data(album_id):
    """ Metodo que genera la lista de ids de canciones por album """
    idList = [item['id'] for item in sp.album(album_id)['tracks']['items']]
    return [get_track_data(idList[item]) for item in range(len(idList))]


def playlist_get_tracks_data(playlist_id):
    """ Metodo que genera la lista de ids de canciones por playlist """
    idList = [item['track']['id']
              for item in sp.playlist(playlist_id)['tracks']['items']]
    return [get_track_data(idList[item]) for item in range(len(idList))]


def get_track_data(id):
    """ Metodo que obtiene los atributos de una cancion """
    # meta
    get_meta = sp.track(id)
    name = get_meta['name']
    album = get_meta['album']['name']
    artist = get_meta['album']['artists'][0]['name']
    release_date = get_meta['album']['release_date']
    length = time_format(int(get_meta['duration_ms']))
    popularity = get_meta['popularity']

    # features
    get_features = sp.audio_features(id)
    danceability = get_features[0]['danceability']
    acousticness = get_features[0]['acousticness']
    energy = get_features[0]['energy']
    instrumentalness = get_features[0]['instrumentalness']
    liveness = get_features[0]['liveness']
    loudness = get_features[0]['loudness']
    speechiness = get_features[0]['speechiness']
    key = get_features[0]['key']
    mode = get_features[0]['mode']
    valence = get_features[0]['valence']
    tempo = get_features[0]['tempo']
    time_signature = get_features[0]['time_signature']

    return [name, album, artist, release_date, length, popularity, danceability, acousticness, energy,
            instrumentalness, liveness, loudness, speechiness, key, mode, valence, tempo, time_signature]


def get_album_data(album_id):
    """ Metodo que obtiene los atributos de un Album """
    get_data = sp.album(album_id)
    albumDict = {
        'Album': get_data['name'],
        'Artista': get_data['artists'][0]['name'],
        'Canciones': get_data['total_tracks'],
        'Duracion Total': time_format(sum([x['duration_ms'] for x in get_data['tracks']['items']])),
        'Lanzamiento': get_data['release_date'],
        'Popularidad': get_data['popularity'],
        'Album Label': get_data['label'],
        'Mercados': len(get_data['available_markets']),
        'Cover': get_data['images'][1]['url'],
    }
    return albumDict


def get_playlist_data(playlist_id):
    """ Metodo que obtiene los atributos de una playlist """
    get_data = sp.playlist(playlist_id)
    playlistDict = {
        'Playlist': get_data['name'],
        'Canciones': get_data['tracks']['total'],
        'Seguidores': get_data['followers']['total'],
        'Duracion Total': time_format(sum([x['track']['duration_ms'] for x in get_data['tracks']['items']])),
        'Popularidad Media': sum([x['track']['popularity'] for x in get_data['tracks']['items']]) / get_data['tracks']['total'],
        'Creador': get_data['owner']['display_name'],
        'Publica': (lambda x: 'Si' if x else 'No')(get_data['public']),
        'Colaboracion': (lambda x: 'Si' if x else 'No')(get_data['collaborative']),
        'Descripcion': get_data['description'],
        'Cover': get_data['images'][0]['url']
    }
    return playlistDict


def crearCSV(dataframe):
    try:
        dataframe.to_csv("spotify.csv", sep=';', encoding='utf-8')
        print("El CSV esta listo.")
    except Exception as e:
        print(e)


In [3]:
#  ID's de Prueba
album_id_test = '0ZgTSm1VI55AhE09Nzvv11'
playlist_id_test = '37i9dQZF1DXagUeYbNSnOA'

In [4]:
#  Respuesta de API (JSON)
nsg = sp.album(album_id_test) # ALBUM
pl = sp.playlist(playlist_id_test) # PLAYLIST
lista_ids = [item['id'] for item in sp.album(album_id_test)['tracks']['items']] # Lista de Tracks ID del Album
id_track5 = sp.album(album_id_test)['tracks']['items'][0]['id'] # Track ID: Como Te Llamas
dictAlbum = {i: time_format(item['duration_ms']) for (i, item) in enumerate(sp.album(album_id_test)['tracks']['items'])} # Diccionario de nombres de canciones del Album
print(pl['tracks'].keys())

for x in pl['tracks']['items']:
    print(x['track'].keys())



dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])
dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'episode', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track', 'track_number', 'type', 'uri'])
dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'episode', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track', 'track_number', 'type', 'uri'])
dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'episode', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track', 'track_number', 'type', 'uri'])
dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'episode', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_ur

In [5]:
# DataFrame
df = pd.DataFrame(album_get_tracks_data(album_id_test), columns=meta+features)

df.Name = f"{sp.album(album_id_test)['name']} - {sp.album(album_id_test)['artists'][0]['name']}"
df.rename(columns={"duration_ms": "duration"}, inplace=True)
# df.drop(['time_signature', 'valence', 'album', 'artists', 'release_date'], axis=1 ,inplace=True)

In [6]:
# Tabla
df

,name,album,artists,release_date,duration,popularity,danceability,acousticness,energy,instrumentalness,liveness,loudness,speechiness,key,mode,valence,tempo,time_signature
0,Despierta - Remastered 2021,Noches Sin Gravedad,Los Nafta,2022-01-14,03:45,1,0.378,0.3250,0.643,0.01150,0.1630,-8.209,0.0452,4,1,0.117,139.909,4
1,Prólogo,Noches Sin Gravedad,Los Nafta,2022-01-14,01:25,0,0.606,0.8190,0.708,0.88000,0.1880,-12.578,0.0625,11,0,0.172,122.099,4
2,Helado - Remastered 2021,Noches Sin Gravedad,Los Nafta,2022-01-14,03:50,0,0.504,0.3310,0.684,0.65700,0.1980,-7.870,0.0512,11,0,0.270,143.036,4
3,Tu Secreto,Noches Sin Gravedad,Los Nafta,2022-01-14,03:57,1,0.445,0.3420,0.640,0.34900,0.1820,-8.908,0.0368,9,0,0.478,159.999,4
4,¿Cómo Te Llamas?,Noches Sin Gravedad,Los Nafta,2022-01-14,03:15,4,0.420,0.4570,0.570,0.03400,0.2040,-8.990,0.0403,10,0,0.352,155.066,4
5,Noches Sin Gravedad,Noches Sin Gravedad,Los Nafta,2022-01-14,03:34,1,0.408,0.1470,0.669,0.11300,0.0981,-9.066,0.1350,4,0,0.387,143.186,4
6,Nademos en el Fuego - Remastered 2021,Noches Sin Gravedad,Los Nafta,2022-01-14,02:50,1,0.534,0.0554,0.711,0.09180,0.1450,-7.432,0.0405,9,0,0.474,138.025,4
7,Muro de Piedra - Remastered 2021,Noches Sin Gravedad,Los Nafta,2022-01-14,03:50,0,0.551,0.3750,0.636,0.00567,0.1280,-7.564,0.0559,5,0,0.465,119.955,4
8,Tu Mal,Noches Sin Gravedad,Los Nafta,2022-01-14,00:46,0,0.407,0.7730,0.509,0.97200,0.3870,-10.006,0.0341,11,0,0.235,112.725,4
9,Última Píldora,Noches Sin Gravedad,Los Nafta,2022-01-14,03:15,1,0.544,0.1710,0.796,0.01250,0.0912,-7.882,0.1280,11,0,0.438,120.002,4


In [7]:
albumData = get_album_data(album_id_test)
list(albumData.keys())
list(albumData.values())

df_info = pd.DataFrame(albumData.values(), index=list(albumData.keys()), columns=['Info'])
df_info

,Info
Album,Noches Sin Gravedad
Artista,Los Nafta
Canciones,12
Duracion Total,37:33
Lanzamiento,2022-01-14
Popularidad,2
Album Label,Los Nafta
Mercados,183
Cover,https://i.scdn.co/image/ab67616d00001e027ce9b9...


In [8]:
from spotipy import SpotifyException
try:    
    test = sp.playlist('37824782bhjde')
except (SpotifyException) as e:
    print(e)
    


http status: 404, code:-1 - https://api.spotify.com/v1/playlists/37824782bhjde?additional_types=track:
 Invalid playlist Id, reason: None
